# Código para descargar las imagenes de Sentinel-2 desde ESA Copernicus program.
## Se descargan los resultados mResultado se guarda en .PNG

# Configuracion

Estos primeros bloques son de importar librerías, y configurar las credenciales de la API para poder descargar

In [1]:
# Codificación Rodrigo Pereira
# rodrigo.pereira.r@mail.pucv.cl
# Laboratorio de Robótica y Visión, EIE PUCV

In [2]:
# Importar librerías
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import matplotlib.image as mpimg
from PIL import Image  
import tqdm as notebook_tqdm

In [3]:
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubCatalog,
    Geometry
)

/home/noobird/anaconda3/envs/fondecyt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sentinelhub import SHConfig

config = SHConfig()
# config.sh_client_id = "259c05ca-65df-4c02-bd31-0bc10e469ab0"
config.sh_client_id = "sh-59d7f122-872a-4f74-ab35-826730be486a"

# config.sh_client_secret = "Qdo9JpGqxSZf2Xh2YzWS6o15j5m6YNTS"
config.sh_client_secret = "GGiPWv1NMfRHNj7PWRwy3GRNVDLQIrx0"
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

config = SHConfig("cdse")

config = SHConfig()

# Ingresar client id
config.sh_client_id = "259c05ca-65df-4c02-bd31-0bc10e469ab0"

# Ingresar client secret 
config.sh_client_secret = "Qdo9JpGqxSZf2Xh2YzWS6o15j5m6YNTS"
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.save("cdse")

config = SHConfig("cdse")

# Area

Estos bloques son para asignar el área geográfica para descargar. Son Bounding Boxs rectangulares con las coordenadas WGS84

El formato es: Coordenadas Esquina Inferior Izquierda, Coordenadas Esquina Superior Derecha

Se elige también la resolución en metros/pixel y se muestra el tamaño (resolución en pixeles) aproximado de la imagen a descargar. 

> LA API NO ADMITE TAMAÑOS DE IMÁGENES MAYORES A 2500 PIXELES DE ALTO O ANCHO

Las coordenadas están hechas de tal forma que ninguna imagen queda mayor a 2500 pixles por ningún lado.

In [5]:
# Specify the path to your Excel file and the name of the sheet
excel_file = 'dbCoordenadas.xlsx'
sheet_name = 'Sheet1'

# Read the specific column from the Excel file into a DataFrame
column_name = 'B'  # Replace 'A' with the column letter or name you want to read
df = pd.read_excel(excel_file, sheet_name=sheet_name, usecols=column_name)

df['Bbox'] = df['Bbox'].str.slice(start=7)
df['Bbox'] = df['Bbox'].str[:-1]

# Split the string numbers into multiple columns
df[['longMin', 'latMin', 'longMax', 'latMax']] = df['Bbox'].str.split(',', expand=True)

# Convert the new columns to numeric
df[['longMin', 'latMin', 'longMax', 'latMax']] = df[['longMin', 'latMin', 'longMax', 'latMax']].apply(pd.to_numeric)

# Drop the original column
df.drop(columns=['Bbox'], inplace=True)


listaDeListas = df.values.tolist()

In [6]:
for coordenadas in listaDeListas:

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
    
    print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')
    if aoi_size[0] > 2500 or aoi_size[1] > 2500:
        print(f'Image shape at {coordenadas} m resolution: {aoi_size} pixels')

Image shape at [-68.769951, -21.022502, -68.581467, -20.926971] m resolution: (1961, 1053) pixels
Image shape at [-68.768406, -21.020739, -68.660259, -20.929697] m resolution: (1126, 1006) pixels
Image shape at [-68.8484, -21.032276, -68.767548, -20.97795] m resolution: (841, 600) pixels
Image shape at [-69.317894, -20.095433, -69.223137, -20.018355] m resolution: (989, 855) pixels
Image shape at [-68.864365, -21.941931, -68.782654, -21.896544] m resolution: (844, 501) pixels
Image shape at [-68.73991, -22.033616, -68.642578, -21.958012] m resolution: (1006, 835) pixels
Image shape at [-68.808746, -22.153862, -68.971481, -22.2875] m resolution: (1678, 1478) pixels
Image shape at [-68.808746, -22.2875, -68.971481, -22.421185] m resolution: (1677, 1479) pixels
Image shape at [-69.678211, -22.348806, -69.614182, -22.305614] m resolution: (657, 481) pixels
Image shape at [-69.642334, -22.419598, -69.591179, -22.367698] m resolution: (524, 577) pixels
Image shape at [-69.704647, -22.646017,

# Scripts

Estos scripts se usan para descirle a la API lo que tiene que descargar.

En este caso, son Todas las bandas del satelite.

In [7]:
# Script todo

evalscriptAllBands = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: [
          "B01",
          "B02",
          "B03",
          "B04",
          "B05",
          "B06",
          "B07",
          "B08",
          "B8A",
          "B09",
          "B11",
          "B12",
        ],
        units: "DN",
      },
    ],
    output: {
      id: "default",
      bands: 12,
      sampleType: SampleType.UINT16,
    },
  }
}

function evaluatePixel(sample) {
  return [
    sample.B01,
    sample.B02,
    sample.B03,
    sample.B04,
    sample.B05,
    sample.B06,
    sample.B07,
    sample.B08,
    sample.B8A,
    sample.B09,
    sample.B11,
    sample.B12,
  ]
}
"""

# Rango de Tiempo
Este bloque determina el rango de tiempo a descargar. Originalmente está en 2019, 1, 1 - 2024, 4,3
Se deben definir cuántas imágenes descargar en dicho tiempo, por ejemplo 50 imágenes entre el 2019 - 2024

In [8]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2024, 4, 3)
n_chunks = 50
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks+1)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

Monthly time windows:

('2019-01-01', '2019-02-08')
('2019-02-08', '2019-03-18')
('2019-03-18', '2019-04-26')
('2019-04-26', '2019-06-03')
('2019-06-03', '2019-07-11')
('2019-07-11', '2019-08-19')
('2019-08-19', '2019-09-26')
('2019-09-26', '2019-11-04')
('2019-11-04', '2019-12-12')
('2019-12-12', '2020-01-19')
('2020-01-19', '2020-02-27')
('2020-02-27', '2020-04-05')
('2020-04-05', '2020-05-13')
('2020-05-13', '2020-06-21')
('2020-06-21', '2020-07-29')
('2020-07-29', '2020-09-06')
('2020-09-06', '2020-10-14')
('2020-10-14', '2020-11-21')
('2020-11-21', '2020-12-30')
('2020-12-30', '2021-02-06')
('2021-02-06', '2021-03-16')
('2021-03-16', '2021-04-24')
('2021-04-24', '2021-06-01')
('2021-06-01', '2021-07-10')
('2021-07-10', '2021-08-17')
('2021-08-17', '2021-09-24')
('2021-09-24', '2021-11-02')
('2021-11-02', '2021-12-10')
('2021-12-10', '2022-01-18')
('2022-01-18', '2022-02-25')
('2022-02-25', '2022-04-04')
('2022-04-04', '2022-05-13')
('2022-05-13', '2022-06-20')
('2022-06-20', '2022

# Descargar
El primer bloque es para procesamiento de imágenes.
El factor es principalmente para el brillo de la imagen. Estaba originalmente en 2 / 1e4, que es un buen factor para el espectro visible.

In [9]:
factor = 2 / 1e4
clipRange= (0, 1)

In [13]:
for coordenadas in listaDeListas:

    resolution = 10
    aoi_bbox = BBox(bbox=coordenadas, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

    for slot in slots:
        request_all_bands = SentinelHubRequest(
                evalscript= evalscriptAllBands,
                input_data=[
                    SentinelHubRequest.input_data(
                            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                                name="s2", service_url="https://sh.dataspace.copernicus.eu"
                            ),
                            time_interval=slot,
                            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}           )
                    ],
                responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
                bbox=aoi_bbox,
                size=aoi_size,
                config=config,
            )
        all_bands_img = request_all_bands.get_data()

        # Carpetas
        nombreCarpetas = f'{coordenadas}'

        if not os.path.exists(nombreCarpetas):
            os.makedirs(nombreCarpetas)
    
        # Bandas RAW sin procesar
    
        bandaB01 = (np.clip(all_bands_img[0][:,:,0]*factor,*clipRange))*255
        bandaB01 = bandaB01.astype(np.uint8)
        bandaB01 = Image.fromarray(bandaB01)
        bandaB01.save(f'{nombreCarpetas}/{slot} B01.png')
    
        bandaB02 = (np.clip(all_bands_img[0][:,:,1]*factor,*clipRange))*255
        bandaB02 = bandaB02.astype(np.uint8)
        bandaB02 = Image.fromarray(bandaB02)
        bandaB02.save(f'{nombreCarpetas}/{slot} B02.png')
    
        bandaB03 = (np.clip(all_bands_img[0][:,:,2]*factor,*clipRange))*255
        bandaB03 = bandaB03.astype(np.uint8)
        bandaB03 = Image.fromarray(bandaB03)
        bandaB03.save(f'{nombreCarpetas}/{slot} B03.png')
    
        bandaB04 = (np.clip(all_bands_img[0][:,:,3]*factor,*clipRange))*255
        bandaB04 = bandaB04.astype(np.uint8)
        bandaB04 = Image.fromarray(bandaB04)
        bandaB04.save(f'{nombreCarpetas}/{slot} B04.png')
    
        bandaB05 = (np.clip(all_bands_img[0][:,:,4]*factor,*clipRange))*255
        bandaB05 = bandaB05.astype(np.uint8)
        bandaB05 = Image.fromarray(bandaB05)
        bandaB05.save(f'{nombreCarpetas}/{slot} B05.png')
    
        bandaB06 = (np.clip(all_bands_img[0][:,:,5]*factor,*clipRange))*255
        bandaB06 = bandaB06.astype(np.uint8)
        bandaB06 = Image.fromarray(bandaB06)
        bandaB06.save(f'{nombreCarpetas}/{slot} B06.png')
    
        bandaB07 = (np.clip(all_bands_img[0][:,:,6]*factor,*clipRange))*255
        bandaB07 = bandaB07.astype(np.uint8)
        bandaB07 = Image.fromarray(bandaB07)
        bandaB07.save(f'{nombreCarpetas}/{slot} B07.png')
    
        bandaB08 = (np.clip(all_bands_img[0][:,:,7]*factor,*clipRange))*255
        bandaB08 = bandaB08.astype(np.uint8)
        bandaB08 = Image.fromarray(bandaB08)
        bandaB08.save(f'{nombreCarpetas}/{slot} B08.png')
    
        bandaB8A = (np.clip(all_bands_img[0][:,:,8]*factor,*clipRange))*255
        bandaB8A = bandaB8A.astype(np.uint8)
        bandaB8A = Image.fromarray(bandaB8A)
        bandaB8A.save(f'{nombreCarpetas}/{slot} B8A.png')
    
        bandaB09 = (np.clip(all_bands_img[0][:,:,9]*factor,*clipRange))*255
        bandaB09 = bandaB09.astype(np.uint8)
        bandaB09 = Image.fromarray(bandaB09)
        bandaB09.save(f'{nombreCarpetas}/{slot} B09.png')
    
        bandaB11 = (np.clip(all_bands_img[0][:,:,10]*factor,*clipRange))*255
        bandaB11 = bandaB11.astype(np.uint8)
        bandaB11 = Image.fromarray(bandaB11)
        bandaB11.save(f'{nombreCarpetas}/{slot} B11.png')
    
        bandaB12 = (np.clip(all_bands_img[0][:,:,11]*factor,*clipRange))*255
        bandaB12 = bandaB12.astype(np.uint8)
        bandaB12 = Image.fromarray(bandaB12)
        bandaB12.save(f'{nombreCarpetas}/{slot} B12.png')
    
    
        # Bandas Procesadas
    
        trueColor = (np.clip(all_bands_img[0][:,:,[3,2,1]]*factor,*clipRange))*255
        trueColor = trueColor.astype(np.uint8)
        trueColor = Image.fromarray(trueColor)
        trueColor.save(f'{nombreCarpetas}/{slot} - trueColor.png')
   
        #falseColor = (np.clip(all_bands_img[0][:,:,[7,3,2]]*factor,*clipRange))*255
        #falseColor = falseColor.astype(np.uint8)
        #falseColor = Image.fromarray(falseColor)
        #falseColor.save(f'{slot} falseColor.png')




DownloadFailedException: Failed to download from:
https://sh.dataspace.copernicus.eu/api/v1/process
with HTTPError:
500 Server Error: Internal Server Error for url: https://sh.dataspace.copernicus.eu/api/v1/process
Server response: "{"status": 500, "reason": "Internal Server Error", "message": "Illegal request to creo://EODATA/Sentinel-2/MSI/L2A/2019/01/19/S2A_MSIL2A_20190119T143751_N0211_R096_T19KES_20190119T184849.SAFE/GRANULE/L2A_T19KES_A018683_20190119T144606/IMG_DATA/R10m/T19KES_20190119T143751_B03_10m.jp2. HTTP Status: '404'", "code": "RENDERER_EXCEPTION"}"

In [ ]:
trueColor.save(f'{nombreCarpetas}/{slot} - trueColor.png')

In [ ]:
import rasterio
from rasterio.transform import from_origin

# Suponiendo que `all_bands_img[0][:,:,1]` es tu banda B02 y conoces sus dimensiones
bandaB02 = np.clip(all_bands_img[0][:,:,1]*factor,*clipRange)

# Suponiendo que conoces el transform (geotransform) y el sistema de coordenadas (CRS)
transform = from_origin(west, north, pixel_width, pixel_height) # Estos valores deben ser definidos según tu caso específico
crs = "EPSG:4326" # Por ejemplo, WGS84

with rasterio.open(f'{nombreCarpetas}/{slot} B02.tif', 'w', driver='GTiff',
                   height=bandaB02.shape[0], width=bandaB02.shape[1],
                   count=1, dtype=str(bandaB02.dtype),
                   crs=crs, transform=transform) as dst:
    dst.write(bandaB02, 1)


In [ ]:
import rasterio
from rasterio.transform import from_origin

# Primero, ajusta tus datos como lo haces normalmente
bandaB02 = (np.clip(all_bands_img[0][:,:,1]*factor, *clipRange))*255
bandaB02 = bandaB02.astype(np.uint8)

bandaB03 = (np.clip(all_bands_img[0][:,:,2]*factor, *clipRange))*255
bandaB03 = bandaB03.astype(np.uint8)

# Asumiendo que conoces las dimensiones y la resolución espacial de tus datos, define el transform
west, north = -70.665, -33.436  # Estos son ejemplos. Debes utilizar tus propias coordenadas
pixel_width, pixel_height = 10, -10  # Asumiendo una resolución de 10x10 metros por píxel
transform = from_origin(west, north, pixel_width, pixel_height)

# Guarda la banda como TIFF con rasterio
def save_band_as_tiff(band_data, band_name, folder, slot, transform, crs='EPSG:4326'):
    # Define las dimensiones y el tipo de datos de la imagen
    height, width = band_data.shape
    dtype = band_data.dtype

    # Configura los metadatos del archivo TIFF
    metadata = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,
        'dtype': dtype,
        'crs': crs,
        'transform': transform
    }

    with rasterio.open(f'{folder}/{slot} {band_name}.tif', 'w', **metadata) as dst:
        dst.write(band_data, 1)

# Ahora llama a la función para guardar cada banda
save_band_as_tiff(bandaB02, 'B02', nombreCarpetas, slot, transform)
save_band_as_tiff(bandaB03, 'B03', nombreCarpetas, slot, transform)
